# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
engine = create_engine('sqlite:///disaster_message.db')
df = pd.read_sql_table('tbl_disaster_message', 'sqlite:///disaster_message.db')  
X = df.loc[:,'message'] 
Y = df.iloc[:, 4:] 

In [5]:
Y.related.value_counts()

1    20093
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    #: tokenize text
    tokens= word_tokenize(text)
    
    #: initiate Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #: iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        #: Lemmatize, normalize case, and remove Leading/trailing wite space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_predict = pipeline.predict(X_test)

In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4999
               request       0.87      0.42      0.57      1093
                 offer       0.00      0.00      0.00        38
           aid_related       0.78      0.60      0.68      2708
          medical_help       0.66      0.04      0.07       506
      medical_products       0.75      0.07      0.13       333
     search_and_rescue       0.86      0.03      0.06       185
              security       0.00      0.00      0.00       119
              military       1.00      0.03      0.05       227
           child_alone       0.00      0.00      0.00         0
                 water       0.95      0.24      0.38       390
                  food       0.87      0.36      0.51       744
               shelter       0.90      0.21      0.35       596
              clothing       0.67      0.02      0.04        86
                 money       0.40      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0

In [49]:
from sklearn.model_selection import GridSearchCV

parameters ={
        'clf__estimator__n_estimators': [10, 50]
        #'clf__estimator__min_samples_split': [2, 10],
        #'clf__estimator__min_samples_leaf': [1, 4]
        #'clf__estimator__max_depth': [20, None]
    }


In [50]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, n_jobs = -1, verbose = 3)

In [51]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vector',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        pr

In [52]:
improved_model = cv.best_params_
improved_model

{'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [53]:
y_predict = cv.predict(X_test)

In [54]:
print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4999
               request       0.87      0.43      0.57      1093
                 offer       0.00      0.00      0.00        38
           aid_related       0.79      0.58      0.67      2708
          medical_help       0.60      0.03      0.06       506
      medical_products       0.86      0.08      0.14       333
     search_and_rescue       0.67      0.02      0.04       185
              security       1.00      0.01      0.02       119
              military       0.78      0.03      0.06       227
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.26      0.40       390
                  food       0.86      0.40      0.55       744
               shelter       0.89      0.24      0.38       596
              clothing       0.75      0.03      0.07        86
                 money       0.25      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

pipeline = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
     ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

In [58]:
parameters ={
        'clf__estimator__learning_rate': [0.2, 0.3],
    }

pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                          class_weight='balanced',
                                                                                          criterion='gini',
                                                                                          max_depth=1,
                                                                                          max_features=None,
                                                                                          max_leaf_nodes=None,
                                                                                          min_impurity_decrease=0.0,
                                                                                          min_impurity_split=None,
                                     

In [59]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3,verbose = 3)

In [60]:
cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vector',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        pr

In [61]:
y_predict = cv.predict(X_test)

In [62]:
print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.92      0.63      0.75      4999
               request       0.53      0.76      0.63      1093
                 offer       0.01      0.37      0.03        38
           aid_related       0.71      0.63      0.67      2708
          medical_help       0.39      0.59      0.47       506
      medical_products       0.23      0.69      0.35       333
     search_and_rescue       0.12      0.49      0.19       185
              security       0.04      0.54      0.07       119
              military       0.33      0.71      0.45       227
           child_alone       0.00      0.00      0.00         0
                 water       0.55      0.88      0.67       390
                  food       0.76      0.82      0.79       744
               shelter       0.59      0.75      0.66       596
              clothing       0.18      0.64      0.28        86
                 money       0.26      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

### 9. Export your model as a pickle file

In [ ]:
model_filepath = 'classifier.pickle'

with open(model_filepath, 'wb') as model_file:
        pickle.dump(model, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.